# **🧪  Lab 1 - Prototype**

This notebook is about building a simple Microsoft Fabric-based Copilot application

### **🔥 Prerequirement**

1. [Visual Studio Code](https://code.visualstudio.com/) or [GitHub Codespaces](https://github.com/features/codespaces)
2. [Azure](https://azure.com/free) or [Azure for Student](https://aka.ms/studentgetazure)
3. Apply [Azure OpenAI Service](https://customervoice.microsoft.com/Pages/ResponsePage.aspx?id=v4j5cvGGr0GRqy180BHbR7en2Ais5pxKtso_Pz4b1_xUOFA5Qk1UWDRBMjg0WFhPMkIzTzhKQ1dWNyQlQCN0PWcu)
4. [.NET 7+](https://dotnet.microsoft.com/en-us/)
5. [Docker](https://www.docker.com/)
6. [Qdrant](https://qdrant.tech/)

### **📚 Intro**

We use Embeddings to inject relevant knowledge into the model of Azure OpenAI Services through the vector database. This is how we infuse magical skills. Learn from the following

Introduce the related library of Semantic Kernel, and introduce the support of Semantic Kernel for Qdrant vector database

In [1]:
#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Qdrant, *-*"

Installed Packages Microsoft.SemanticKernel, 1.0.0-rc2 Microsoft.SemanticKernel.Connectors.Memory.Qdrant, 1.0.0-rc2

In [2]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.Memory.Qdrant;
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.TextEmbedding;

In [3]:
using System;
using System.Collections.Generic;
using System.Diagnostics;
using System.IO;
using System.Net.Http;
using System.Text.Json;
using System.Threading.Tasks;

A kernel in Semantic Kernel (SK) is an orchestrator of user problems. The kernel combines skills, memory and connectors to realize the user's intelligent expectations. In addition to configuring basic Azure OpenAI / OpenAI basic link strings / models / parameters / vector database, it can also pair related skills based on user requirements, integrate skills to form workflows, etc. 

The following example is to bind the Sementic Kernel to the gpt-3.5-turbo-16k and text-embedding-ada-002 models in Azure OpenAI Service, and connect to the locally assumed Qdrant vector database.

**Note**

*1. You need to bind the model of Azure OpenAI Service, which corresponds to the model you deployed in Azure AI Studio*

*2. Azure OpenAI Service Endpoint and Key need to be obtained in Azure Portal*

*3. The dimension of Azure OpenAI Service Embeddings is 1536, which is the default value, and you need to start the Qdrant service before this, please read [here](https://qdrant.tech/)*



In [4]:
Kernel kernel = new KernelBuilder()
            .WithAzureOpenAIChatCompletion(deploymentName:"Your Azure OpenAI Services gpt-35-turbo-16k Deployment Name",modelId:"gpt-35-turbo-16k","Your Azure OpenAI Services Endpoint","Your Azure OpenAI Services API Key")
            .Build();

# **🔑 Key: Chunkings**

With the expansion of model tokens by Azure OpenAI Service, chunking is no longer a problem, but we need to organize knowledge points is also an important step, Microsoft Fabric documents, headers and footers can be removed

In [5]:
string markdownFile = @"../docs/microsoft-fabric-overview.md";
//string markdownFile = @"../docs/data-science-overview.md";
// string markdownFile = @"../docs/data-engineering-overview.md";
// string markdownFile = @".../docs/data-factory-overview.md";
// string markdownFile = @"../docs/data-warehousing.md";
// string markdownFile = @"../docs/fabric-terminology.md";
//string markdownFile = @"../docs/end-to-end-tutorials.md";


In [6]:
string learnContent = File.ReadAllText(markdownFile);

In [7]:
int content_start = learnContent.IndexOf("# ");
int content_end = learnContent.IndexOf("## Next steps");
learnContent = learnContent.Substring(content_start,content_end - content_start) ;  

In [8]:
int chunkSize = learnContent.Length / 600;

In [9]:
var skillsDirectory = Path.Combine("..", "Plugins");

In [10]:
skillsDirectory

../Plugins

***Import Skill into the kernel, and inject magic through Prompt to extract relevant knowledge points***

In [11]:
var read_plugin = kernel.CreatePluginFromPromptDirectory(Path.Combine(skillsDirectory, "ReadPlugin"));


In [12]:
public class KBContent
{
    public string KB { get; set; }
    public string Content { get; set; }
}


In [13]:
read_plugin

[ {"Name":"KB","Description":"Get KB Content","Metadata":{"Name":"KB","PluginName":null,"Description":"Get KB Content","Parameters":[],"ReturnParameter":{"Description":"","ParameterType":null,"Schema":null}}} ] Name ReadPlugin Description FunctionCount 1 (values) index value 0 {"Name":"KB","Description":"Get KB Content","Metadata":{"Name":"KB","PluginName":null,"Description":"Get KB Content","Parameters":[],"ReturnParameter":{"Description":"","ParameterType":null,"Schema":null}}} Name KB Description Get KB Content Metadata Microsoft.SemanticKernel.KernelFunctionMetadata Name KB PluginName <null> Description Get KB Content Parameters (empty) ReturnParameter Microsoft.SemanticKernel.KernelReturnParameterMetadata Description ParameterType <null> Schema <null>

In [14]:
public async Task<string> GetKBContent(Kernel kernel,string content)
{
    var kbContent = await kernel.InvokeAsync(read_plugin["KB"],new(content));

    return kbContent.ToString();
}

In [15]:
bool checkStr = false;
string kbContent = "";
var kbList = new List<KBContent>();

In [16]:
kbContent = await GetKBContent(kernel,learnContent.Replace("\\","\\\\").Replace("\"","\'").Replace(":::","").Replace("\n",""));

In [17]:
//string setKBContent = kbContent.Replace("[OUTPUT]","").Replace("[END OUTPUT]","").Trim();

Through Prompt, we extract the content of knowledge points into JSON format, which is convenient for us to import into the relevant vector database

In [18]:
int pos = 0;
if(kbContent.IndexOf("[OUTPUT]")>-1)
    pos =  kbContent.IndexOf("[OUTPUT]");
string setKBContent = kbContent.Substring(pos).Replace("[OUTPUT]","").Replace("[END OUTPUT]","").Trim();

In [19]:
setKBContent

[{"KB": "What is Microsoft Fabric?", "Content": "Microsoft Fabric is an all-in-one analytics solution for enterprises that covers everything from data movement to data science, Real-Time Analytics, and business intelligence. It offers a comprehensive suite of services, including data lake, data engineering, and data integration, all in one place.With Fabric, you don't need to piece together different services from multiple vendors. Instead, you can enjoy a highly integrated, end-to-end, and easy-to-use product that is designed to simplify your analytics needs.The platform is built on a foundation of Software as a Service (SaaS), which takes simplicity and integration to a whole new level."}, {"KB": "SaaS foundation", "Content": "Microsoft Fabric brings together new and existing components from Power BI, Azure Synapse, and Azure Data Explorer into a single integrated environment. These components are then presented in various customized user experiences.Fabric brings together experience

In [20]:
var jsonKBContent = System.Text.Json.JsonSerializer.Deserialize<List<KBContent>>(setKBContent);

In [21]:
foreach(var item in jsonKBContent)
{
    kbList.Add(item);
}

The token is added or we don’t need to split the content, but sometimes there are unexpected situations, you need to intercept the content, segment and interact with the model

In [22]:
// for(int i = 0; i < chunkSize+1; i++)
// {
                 
//     if(checkStr)
//     {
//                //Console.WriteLine(strTmp);
//         kbContent = await GetKBContent(kernel,learnContent.Substring(i*600).Replace("\\","\\\\").Replace("\"","\'").Replace(":::",""));
//     }
//     else
//     {
//         // strTmp = saveContent.Substring(i*600);
//                //Console.WriteLine(strTmp.Substring(0,600));
//         kbContent = await GetKBContent(kernel,learnContent.Substring(i*600,600).Replace("\\","\\\\").Replace("\"","\'").Replace(":::",""));
//     }


//     string setKBContent = kbContent.Replace("[OUTPUT]","").Replace("[END OUTPUT]","").Trim();


//     Console.WriteLine(setKBContent);


//     var jsonKBContent = System.Text.Json.JsonSerializer.Deserialize<List<KBContent>>(setKBContent);

//     foreach(var item in jsonKBContent)
//     {
//         kbList.Add(item);
//     }

//     if(i!=chunkSize)
//     {

//         string strTmp = learnContent.Substring((i+1)*600);

//         if(strTmp.Length <= 600)
//             checkStr = true;
//         else
//             checkStr = false;
//     }

// }

In [23]:
kbList

index value 0 Submission#12+KBContent KB What is Microsoft Fabric? Content Microsoft Fabric is an all-in-one analytics solution for enterprises that covers everything from data movement to data science, Real-Time Analytics, and business intelligence. It offers a comprehensive suite of services, including data lake, data engineering, and data integration, all in one place.With Fabric, you don't need to piece together different services from multiple vendors. Instead, you can enjoy a highly integrated, end-to-end, and easy-to-use product that is designed to simplify your analytics needs.The platform is built on a foundation of Software as a Service (SaaS), which takes simplicity and integration to a whole new level. 1 Submission#12+KBContent KB SaaS foundation Content Microsoft Fabric brings together new and existing components from Power BI, Azure Synapse, and Azure Data Explorer into a single integrated environment. These components are then presented in various customized user experiences.Fabric brings together experiences such as Data Engineering, Data Factory, Data Science, Data Warehouse, Real-Time Analytics, and Power BI onto a shared SaaS foundation. This integration provides the following advantages:- An extensive range of deeply integrated analytics in the industry.- Shared experiences across experiences that are familiar and easy to learn.- Developers can easily access and reuse all assets.- A unified data lake that allows you to retain the data where it is while using your preferred analytics tools.- Centralized administration and governance across all experiences.With the Microsoft Fabric SaaS experience, all the data and the services are seamlessly integrated. IT teams can centrally configure core enterprise capabilities and permissions are automatically applied across all the underlying services. Additionally, data sensitivity labels are inherited automatically across the items in the suite.Fabric allows creators to concentrate on producing their best work, freeing them from the need to integrate, manage, or understand the underlying infrastructure that supports the experience. 2 Submission#12+KBContent KB Components of Microsoft Fabric Content Microsoft Fabric offers the comprehensive set of analytics experiences designed to work together seamlessly. Each experience is tailored to a specific persona and a specific task. Fabric includes industry-leading experiences in the following categories for an end-to-end analytical need.- **Data Engineering** - Data Engineering experience provides a world class Spark platform with great authoring experiences, enabling data engineers to perform large scale data transformation and democratize data through the lakehouse. Microsoft Fabric Spark's integration with Data Factory enables notebooks and spark jobs to be scheduled and orchestrated. For more information, see [What is Data engineering in Microsoft Fabric?](../data-engineering/data-engineering-overview.md)- **Data Factory** - Azure Data Factory combines the simplicity of Power Query with the scale and power of Azure Data Factory. You can use more than 200 native connectors to connect to data sources on-premises and in the cloud. For more information, see [What is Data Factory in Microsoft Fabric?](../data-factory/data-factory-overview.md)- **Data Science** - Data Science experience enables you to build, deploy, and operationalize machine learning models seamlessly within your Fabric experience. It integrates with Azure Machine Learning to provide built-in experiment tracking and model registry. Data scientists are empowered to enrich organizational data with predictions and allow business analysts to integrate those predictions into their BI reports. This way it shifts from descriptive to predictive insights. For more information, see [What is Data science in Microsoft Fabric?](../data-science/data-science-overview.md)- **Data Warehouse** - Data Warehouse experience provides industry leading SQL performance and scale. It 

In [24]:
var vectorData =  new List<KBContent>();

int stepsCount = 0;

foreach(var item in kbList)
{
    if(item.KB!="")
    {
        vectorData.Add(item);
    }
    else
    {
        if(vectorData.Count!=0)
        {
            vectorData[vectorData.Count-1].Content += item.Content;
        }
    }

}

In [25]:
vectorData

index value 0 Submission#12+KBContent KB What is Microsoft Fabric? Content Microsoft Fabric is an all-in-one analytics solution for enterprises that covers everything from data movement to data science, Real-Time Analytics, and business intelligence. It offers a comprehensive suite of services, including data lake, data engineering, and data integration, all in one place.With Fabric, you don't need to piece together different services from multiple vendors. Instead, you can enjoy a highly integrated, end-to-end, and easy-to-use product that is designed to simplify your analytics needs.The platform is built on a foundation of Software as a Service (SaaS), which takes simplicity and integration to a whole new level. 1 Submission#12+KBContent KB SaaS foundation Content Microsoft Fabric brings together new and existing components from Power BI, Azure Synapse, and Azure Data Explorer into a single integrated environment. These components are then presented in various customized user experiences.Fabric brings together experiences such as Data Engineering, Data Factory, Data Science, Data Warehouse, Real-Time Analytics, and Power BI onto a shared SaaS foundation. This integration provides the following advantages:- An extensive range of deeply integrated analytics in the industry.- Shared experiences across experiences that are familiar and easy to learn.- Developers can easily access and reuse all assets.- A unified data lake that allows you to retain the data where it is while using your preferred analytics tools.- Centralized administration and governance across all experiences.With the Microsoft Fabric SaaS experience, all the data and the services are seamlessly integrated. IT teams can centrally configure core enterprise capabilities and permissions are automatically applied across all the underlying services. Additionally, data sensitivity labels are inherited automatically across the items in the suite.Fabric allows creators to concentrate on producing their best work, freeing them from the need to integrate, manage, or understand the underlying infrastructure that supports the experience. 2 Submission#12+KBContent KB Components of Microsoft Fabric Content Microsoft Fabric offers the comprehensive set of analytics experiences designed to work together seamlessly. Each experience is tailored to a specific persona and a specific task. Fabric includes industry-leading experiences in the following categories for an end-to-end analytical need.- **Data Engineering** - Data Engineering experience provides a world class Spark platform with great authoring experiences, enabling data engineers to perform large scale data transformation and democratize data through the lakehouse. Microsoft Fabric Spark's integration with Data Factory enables notebooks and spark jobs to be scheduled and orchestrated. For more information, see [What is Data engineering in Microsoft Fabric?](../data-engineering/data-engineering-overview.md)- **Data Factory** - Azure Data Factory combines the simplicity of Power Query with the scale and power of Azure Data Factory. You can use more than 200 native connectors to connect to data sources on-premises and in the cloud. For more information, see [What is Data Factory in Microsoft Fabric?](../data-factory/data-factory-overview.md)- **Data Science** - Data Science experience enables you to build, deploy, and operationalize machine learning models seamlessly within your Fabric experience. It integrates with Azure Machine Learning to provide built-in experiment tracking and model registry. Data scientists are empowered to enrich organizational data with predictions and allow business analysts to integrate those predictions into their BI reports. This way it shifts from descriptive to predictive insights. For more information, see [What is Data science in Microsoft Fabric?](../data-science/data-science-overview.md)- **Data Warehouse** - Data Warehouse experience provides industry leading SQL performance and scale. It 

## **🔑 Key: VectorDB**

We need to store knowledge in the vector database and query through the vector database. Here is a simple example

In [26]:
string conceptCollectionName = "fbkb-concept";

In [34]:
#pragma warning disable SKEXP0052
#pragma warning disable CS1061
#pragma warning disable SKEXP0011
#pragma warning disable SKEXP0026

var qdrantMemoryBuilder = new MemoryBuilder();

var textEmbedding = new AzureOpenAITextEmbeddingGeneration("Your Azure OpenAI Services text-embedding-ada-002 Deployment Name","text-embedding-ada-002", "Your Azure OpenAI Services Endpoint", "Your Azure OpenAI Services API Key");

qdrantMemoryBuilder.WithTextEmbeddingGeneration(textEmbedding);
qdrantMemoryBuilder.WithQdrantMemoryStore("Your Qdrant endpoint", 1536);

var qdrantBuilder = qdrantMemoryBuilder.Build();

In [35]:
int vectorCount = 1000;

foreach(var item in vectorData)
{
  await qdrantBuilder.SaveInformationAsync(conceptCollectionName, id: "id"+vectorCount.ToString(), text: item.KB+" "+ item.Content);
  vectorCount++;
}

In [36]:
string questionText = "What is Mircorosft Fabric ?"; 

The key to vector search is to find approximate matching vector content

In [37]:
var searchResults =  qdrantBuilder.SearchAsync(conceptCollectionName, questionText, limit: 1, minRelevanceScore: 0.7);

In [38]:

string result = "";
await foreach (var item in searchResults)
{
    result = item.Metadata.Text;
    Console.WriteLine(item.Metadata.Text + " : " + item.Relevance);
}

What is Microsoft Fabric? Microsoft Fabric is an all-in-one analytics solution for enterprises that covers everything from data movement to data science, Real-Time Analytics, and business intelligence. It offers a comprehensive suite of services, including data lake, data engineering, and data integration, all in one place.With Fabric, you don't need to piece together different services from multiple vendors. Instead, you can enjoy a highly integrated, end-to-end, and easy-to-use product that is designed to simplify your analytics needs.The platform is built on a foundation of Software as a Service (SaaS), which takes simplicity and integration to a whole new level. : 0.8197839


## **✅ Summary**

We can inject content knowledge into LLM by combining Chunking and vector database. What we need to consider is how to combine the content with the model for optimization. So what needs to be more is the content of Prompt。 Learn more about [link](https://learn.microsoft.com/en-us/semantic-kernel/prompt-engineering/)